# Adaptive PDE discretizations on cartesian grids : Algorithmic tools

## Part : Automatic differentiation
## Chapter : Dense automatic differentiation, and geodesic shooting

We illustrate dense automatic differentiation, both first and second order, with an application to geodesic shooting.

<!---
**Symplectic integrators**
In this notebook, we solve the equations of geodesics using a standard Runge-Kutta scheme. This may sound as a heresy since these equations benefit from a Hamiltonian structure, and can thus be solved more stably using a symplectic integrator, in principle. However, our Hamiltonians are *non-separable*, in other words they do *not* take the form 
$$
    H(q,p) = F(p) + V(q).
$$
For this reason, the implementation details of a symplectic scheme are non-trivial, and we thus stick to a standard ODE scheme.
--->

In [11]:
import sys; sys.path.append("../..") # Allow imports from parent directory

In [13]:
from Miscellaneous import TocTools; TocTools.displayTOC('Dense','Algo')

[**Summary**](Summary.ipynb) of this series of notebooks. [(view online)](http://nbviewer.jupyter.org/urls/rawgithub.com/Mirebeau/AdaptiveGridDiscretizations/master/Notebooks_Algo/Dense.ipynb)

[**Main summary**](../Summary.ipynb), including the other volumes of this work. [(view online)](http://nbviewer.jupyter.org/urls/rawgithub.com/Mirebeau/AdaptiveGridDiscretizations/master/Summary.ipynb)


# Table of contents

  * [1. Hamilton's equations of geodesics](#1.-Hamilton's-equations-of-geodesics)




**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay


## 0. Importing the required libraries

In [6]:
from NumericalSchemes import AutomaticDifferentiation as ad

In [7]:
import numpy as np

In [14]:
def reload_packages():
    import importlib
    ad = importlib.reload(sys.modules['NumericalSchemes.AutomaticDifferentiation'])
    ad.Sparse = importlib.reload(ad.Sparse)
    ad.Dense = importlib.reload(ad.Dense)
    ad.Sparse2 = importlib.reload(ad.Sparse2)
    ad.Dense2 = importlib.reload(ad.Dense2)

## 1. Automatic differentiation basics

In order to illustrate automatic differentiation, we consider Hamilton's equations of geodesics, reading
$$
    \dot q = \partial_p H(p,q), \qquad \dot p = -\partial_q H(p,q),
$$
where $H$ is the *Hamiltonian* of the medium. Following the classical convention we denote by 
* $q$ the position variable,
* $p$ the impultion, which is dual to the velocity.

**Riemannian Hamiltonian** For a domain $\Omega \subset R^d$ equipped with a Riemannian metric $M : \Omega \to S_d^{++}$, the Hamiltonian is defined as 
$$
    H(q,p) := \frac 1 2 \|p\|_{D(q)}^2, \quad \text{where} D(q) := M(q)^{-1}.
$$

**Poincaré's half plane model.** 
We illustrate this ODE in the case of Poincaré's half plane model of the hyperbolic space, defined by the Riemannian metric
$$
    M(q) := \frac{\rm Id}{q_1^2},
$$
where $q = (q_0,q_1) \in \Omega = R \times R^{++}$. Thus $H(q,p) = (q_1^2/2) \|p\|^2$. 

### 1.1 Differentiating a function

**Implementation note.** The type `numpy.float64` often causes trouble due to bad operator priority (when multiplied with an array of shape `()` and containing automatic differentiation information). We circumvent this issue using the function ad.to_array which casts any value of to a numpy array, in this case to an array containing a single element and of shape $()$ (the empty tuple).

In [124]:
def Hamiltonian(q,p):
    return ad.to_array(0.5*q[1]**2)*(p**2).sum(axis=0)

In [125]:
q = np.array([0.,1.])
p = np.array([1.,1.])

In [126]:
Hamiltonian(q,p)

1.0

In order access the Hamiltonian's derivatives, say with respect to position, we evaluate it on a variable featuring first order information. Said otherwise, we create a variable representing the first order Taylor expansion
$$
    q_{\rm ad} = (q_0+\delta_0,q_1+\delta_1) + O(\|\delta\|^2),
$$
where $\delta = (\delta_0,\delta_1)$ is a symbolic, infinitesimal perturbation.

In [127]:
q_ad = ad.Dense.identity(constant=q)

In [128]:
q_ad # Contains an additional array representing first order information.

denseAD(array([0., 1.]),
array([[1., 0.],
       [0., 1.]]))

The first order information is propagated through the computations, and eventually we obtain the result
$$
    H(q_{\rm ad},p) = H(q,p) + <\nabla_q H(q,p),\delta> + O(\|\delta\|^2).
$$
Note that, for Poincaré's half plane model, one has $\nabla_q H(q,p) = (0,q_1 \|p\|^2)$.

In [129]:
Hamiltonian(q_ad,p)

denseAD(array(1.),array([0., 2.]))

We can similarly differentiate w.r.t. the momentum $p$. Poincaré's half plane model obeys
$\nabla_p H(q,p) = q_1^2 p$.

In [130]:
p_ad = ad.Dense.identity(constant=p)
Hamiltonian(q,p_ad)

denseAD(array(1.),array([1., 1.]))

### 1.2 Differentiating w.r.t several variables

Evaluating $H(q_{\rm ad},p_{\rm ad})$ yields an result may sound unexpected: we obtain the sum $\nabla_q H(q,p)+\nabla_p H(q,p)$ of the gradients, instead of their independent values. Indeed, this follows from the first order Taylor expansion
$$
H(q+\delta,p+\delta) = 
H(q,p) + <\nabla_q H(q,p),\delta> + <\nabla_p H(q,p),\delta> + O(\|\delta\|^2),
$$
where $\delta = (\delta_0,\delta_1)$ is an infinitesimal symbolic perturbation.

In [136]:
reload_packages()

In [137]:
Hamiltonian(q_ad,p_ad)

denseAD(array(1.),array([1., 3.]))

In order to obtain the gradient of $H$ w.r.t the independent variables $q$ and $p$, we need to specify that the corresdponding infinitesimal perturbations are independent. Namely set
$$
    q_{\rm ad} = q+(\delta_0,\delta_1), \quad p_{\rm ad} = p+(\delta_2,\delta_3),
$$
where $(\delta_0,\cdots,\delta_4)$ is four dimensional infinitesimal perturbation.

In [146]:
q_ad = ad.Dense.identity(constant=q,shift=(0,p.size))
p_ad = ad.Dense.identity(constant=p,shift=(q.size,0))

As desired, we obtain the full gradient of $H$, which is the concatenation of $\nabla_q H$ and $\nabla_p H$.

In [150]:
Hamiltonian(q_ad,p_ad)

denseAD(array(1.),array([0., 2., 1., 1.]))

In [151]:
q_ad

denseAD(array([0., 1.]),
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.]]))

In [152]:
p_ad

denseAD(array([1., 1.]),
array([[0., 0., 1., 0.],
       [0., 0., 0., 1.]]))

### 1.3 Vectorization

A common programming style in Python application to numerics is vectorized computations using numpy tensors.
This allows avoiding loops, when some operation has to be repeated a large number of times, with improvements in clarity, flexibility, and speed. 

As an example, we compute $H(q_0,p_0)$, $H(q_1,p_1)$, $H(q_2,p_2)$, where $q_0,q_1,q_2$ and $p_0,p_1,p_2$ are different positions and impulsions. We also differentiate these values w.r.t. the impulsion.

In [191]:
q0 = np.array([0.,1.]); q1 = np.array([1.,1.]); q2 = np.array([-2.,1.]); 
p0 = np.array([1.,1.]); p1 = np.array([1.,0.]); p2 = np.array([-1.,-1.]); 

In [192]:
Hamiltonian(q0,p0), Hamiltonian(q1,p1), Hamiltonian(q2,p2) 

(1.0, 0.5, 1.0)

In [199]:
Ham_seq = [Hamiltonian(q,ad.Dense.identity(constant=p)) for (q,p) in ((q0,p0), (q1,p1), (q2,p2))]
Ham_seq

[denseAD(array(1.),array([1., 1.])),
 denseAD(array(0.5),array([1., 0.])),
 denseAD(array(1.),array([-1., -1.]))]

The vectorized form of these computations is as follows.

In [194]:
q_vec = np.stack( (q0,q1,q2),axis=1)
p_vec = np.stack( (p0,p1,p2),axis=1)

In [178]:
Hamiltonian(q_vec,p_vec)

array([1. , 0.5, 1. ])

In [195]:
p_vec_ad = ad.Dense.identity(constant=p_vec)

However, the automatic differentiation result may not be what is desired. Indeed, the entries of `q_vec` are all considered independent, and we recover a six dimensional gradient vector for each component. This is both impractical and numerically expensive.

In [200]:
Ham_vec = Hamiltonian(q_vec,p_vec_ad)
Ham_vec

denseAD(array([1. , 0.5, 1. ]),
array([[ 1.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0., -1.]]))

In [203]:
print("Sequential result : ",Ham_seq[0])
print("Vectorized result : ",Ham_vec[0])

Sequential result :  denseAD(1.0,[1. 1.])
Vectorized result :  denseAD(1.0,[1. 0. 0. 1. 0. 0.])


In this situation, and in contrast with the previous section, we would like the perturbations to be regarded as dependent. This is achieved with the `shape_factor` parameter, which indicates that the last dimensions should be regarded as vectorized dimensions, with correlated perturbations.

In [211]:
p_vec.shape

(2, 3)

The last dimension can be regarded as 'factored', since it is only introduced for vectorization purposes.

In [212]:
shape_factor = p_vec.shape[-1:]

In [213]:
p_vec_ad = ad.Dense.identity(constant=p_vec, shape_factor=shape_factor)

In [214]:
Ham_vec = Hamiltonian(q_vec,p_vec_ad)
Ham_vec

denseAD(array([1. , 0.5, 1. ]),
array([[ 1.,  1.],
       [ 1.,  0.],
       [-1., -1.]]))

In [215]:
print("Vectorized result (with shape_factor): ",Ham_vec[0])

Vectorized result (with shape_factor):  denseAD(1.0,[1. 1.])


This technique is compatible with differentiation w.r.t. multiple independent variables.

In [218]:
q_vec_ad = ad.Dense.identity(constant=q_vec, shape_factor=shape_factor, shift=(0,len(p_vec)) )
p_vec_ad = ad.Dense.identity(constant=p_vec, shape_factor=shape_factor, shift=(len(q_vec),0) )

In [219]:
Ham_vec = Hamiltonian(q_vec_ad,p_vec_ad)
Ham_vec

denseAD(array([1. , 0.5, 1. ]),
array([[ 0.,  2.,  1.,  1.],
       [ 0.,  1.,  1.,  0.],
       [ 0.,  2., -1., -1.]]))

In [220]:
print("Full gradient w.r.t q0,p0 : ", Ham_vec[0])

Full gradient w.r.t q0,p0 :  denseAD(1.0,[0. 2. 1. 1.])


### 1.4 Second order automatic differentiation

There are situations where second order automatic differentiation is needed. It could be achieved, in principle, by recursive calls to first order differentiation. However, for reasons of performance and convenience, a dedicated class is implemented.

In [226]:
q_ad = ad.Dense2.identity(constant=q)

The hessian of Poincaré's half plane model hamiltonian, w.r.t. position is 
$$
    \nabla_q H(q,p) = \begin{pmatrix} 0 & 0 \\ 0 & \|p\|^2 \end{pmatrix}
$$
and w.r.t. impulsion
$$
    \nabla_p H(q,p) = q_0^2 {\rm Id}.
$$

In [233]:
Hamiltonian(q_ad,p)

denseAD2(array(1.),array([0., 2.]),
array([[0., 0.],
       [0., 2.]]))

In [234]:
p_ad = ad.Dense2.identity(constant=p)

In [235]:
Hamiltonian(q,p_ad)

denseAD2(array(1.),array([1., 1.]),
array([[1., 0.],
       [0., 1.]]))

A joint hessian can be computed. 

In [237]:
q_ad = ad.Dense2.identity(constant=q, shift=(0,p.size))
p_ad = ad.Dense2.identity(constant=p, shift=(q.size,0))

In [244]:
p_ad.coef2.shape

(2, 4, 4)

In [238]:
Hamiltonian(q_ad,p_ad)

ValueError: operands could not be broadcast together with shapes (2,) (2,4) 